# Imports

In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import datetime
import requests

In [12]:
def busca_vagas(nome_vaga):
    driver = webdriver.Chrome()
    driver.get('https://portal.gupy.io/')
    driver.maximize_window()
    try:
        cuidado_golpes = driver.find_element(By.XPATH, '//*[@id="radix-0"]/div[2]/button')
        cuidado_golpes.click()
    except:
        pass
    search = driver.find_element(By.XPATH, '/html/body/div/main/div/div/div[2]/div/div/div/div/div[2]/input')
    search.send_keys(nome_vaga)
    search.send_keys(Keys.ENTER)

    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height


    soup = BeautifulSoup(driver.page_source, 'html.parser')

    driver.quit()

    lista_de_vagas = soup.find_all('div', {'class':'sc-a3bd7ea-0 HCzvP'})

    list_columns = [
            'site_da_vaga',
            'link_site',
            'link_origem',
            'data_publicacao',
            'data_coleta',
            'posicao',
            'titulo_da_vaga',
            'local',
            'modalidade',
            'nome_empresa',
            'contrato',
            'regime',
            'pcd',
            'beneficios',
            'codigo_vaga',
            'descricao',
            'data_expiracao'
    ]

    df_vagas = pd.DataFrame(
            columns = list_columns
        )

    for vaga in lista_de_vagas:
    # vaga = lista_de_vagas[0]
        df_aux = pd.DataFrame(
            columns = list_columns
        )

        df_aux.loc[0,'site_da_vaga'] = 'Gupy'
        df_aux.loc[0,'link_site'] = vaga.findAll('a')[0]['href']
        df_aux.loc[0,'link_origem'] = np.nan
        df_aux.loc[0,'data_publicacao'] = vaga.findAll('p', {'class':'sc-dPyBCJ kyoAxx sc-1db88588-0 inqtnx'})[0].text
        df_aux.loc[0,'data_coleta'] = datetime.datetime.today().strftime('%Y-%m-%d')
        df_aux.loc[0,'posicao'] = nome_vaga
        df_aux.loc[0,'titulo_da_vaga'] = vaga.findAll('h2')[0].text
        df_aux.loc[0,'nome_empresa'] = vaga.findAll('p', {'class':'sc-dPyBCJ kyoAxx sc-a3bd7ea-6 cQyvth'})[0].text
        try: 
            df_aux.loc[0,'local'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[0].text
        except:
            df_aux.loc[0,'local'] = np.nan

        
        try:
            df_aux.loc[0,'modalidade'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[1].text
        except:
            df_aux.loc[0,'modalidade'] = np.nan
        try:
            df_aux.loc[0,'contrato'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[2].text
        except:
            df_aux.loc[0,'contrato'] = np.nan

        df_aux.loc[0, 'regime'] = np.nan

        try:
            df_aux.loc[0,'pcd'] = vaga.findAll('span', {'class',"sc-23336bc7-1 cezNaf"})[3].text
        except:
            df_aux.loc[0,'pcd'] = np.nan


        df_aux.loc[0,'beneficios'] = np.nan

        try:
            response = requests.get(vaga.findAll('a')[0]['href']) # link da vaga
            page = BeautifulSoup(response.text)
            lista_descricao_em_texto = [page.findAll('div', {'data-testid':'text-section'})[i].text for i in range(len(page.findAll('div', {'data-testid':'text-section'})))]
            descricao_completa = '\n'.join(lista_descricao_em_texto)
            df_aux.loc[0,'descricao'] = descricao_completa
        except:
            df_aux.loc[0,'descricao'] = np.nan
        
        df_aux.loc[0, 'data_expiracao'] = np.nan

        df_vagas = pd.concat([df_vagas, df_aux], axis=0, ignore_index=True)
        
    df_vagas = df_vagas.reset_index(drop=True)

    df_vagas.to_excel(f'../data/data_raw/{nome_vaga}_vagas_gupy.xlsx', index=False)

    return df_vagas

In [13]:
lista_vagas = ['Analista de Dados', 'Cientista de Dados', 'Engenheiro de Dados']
df_vagas_full = pd.DataFrame()

for vaga in lista_vagas:
    df_vagas_aux = busca_vagas(vaga)

    df_vagas_full = pd.concat([df_vagas_full, df_vagas_aux], axis = 0)
    
df_vagas_full.reset_index()

df_vagas_full.to_excel('../data/data_raw/vagas_gupy.xlsx', index=False)

## ---------------------------------